<a href="https://colab.research.google.com/github/Amangithubs/nsk-project/blob/main/NSK_AI_final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Document Ingestion

This section covers loading document content and preparing it for processing, including installing necessary libraries and defining the function to load different file types.

In [1]:
!pip install PyPDF2 --quiet # PyPDF2 is needed by PyPDFLoader
!pip install langchain --quiet
!pip install langchain-community --quiet
!pip install pypdf --quiet # Install pypdf explicitly as required by PyPDFLoader
!pip install docx2txt --quiet # Required for Docx2txtLoader

from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
import os

def load_document(file_path):
    """
    Loads content from a document based on its file type using Langchain loaders.

    Args:
        file_path (str): The path to the document file.

    Returns:
        list[str]: A list of page content strings extracted from the document.
        Returns an empty list if the file type is not supported or loading fails.
    """
    _, file_extension = os.path.splitext(file_path)
    file_extension = file_extension.lower()

    documents = []
    try:
        if file_extension == ".pdf":
            loader = PyPDFLoader(file_path)
            documents = loader.load()
        elif file_extension == ".txt":
            loader = TextLoader(file_path)
            documents = loader.load()
        elif file_extension == ".docx":
            loader = Docx2txtLoader(file_path)
            documents = loader.load()
        else:
            print(f"Unsupported file type: {file_extension}")
            return [] # Return empty list for unsupported types

    except Exception as e:
        print(f"Error loading document {file_path}: {e}")
        return [] # Return empty list on error

    # Extract text content from Langchain Document objects
    pages_text = [doc.page_content for doc in documents]
    return pages_text

# Note: The process_document function (in cell iw7IqxGzZzYg9) will need to be updated
# to call this new load_document function instead of load_pdf_pages,
# and to pass the file path correctly.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 4.9 MB/s eta 0:00:00


## Language Model (Ollama) Setup

This section covers downloading, installing, and running the Ollama server and downloading the necessary language models (LLMs) that the RAG pipeline will use for text generation.

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
# Start the Ollama server in the background
# This might require specific handling to keep it running in Colab
# A common approach is to run it as a background process
import subprocess
import time

print("Starting Ollama server...")
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5) # Give the server a few seconds to start
print("Ollama server process started.")
# Note: The server will run in the background as long as this Colab runtime is active.

Starting Ollama server...
Ollama server process started.


In [4]:
# Download an Ollama model (e.g., llama2)
# You can replace 'llama2' with another model name if you prefer
print("Downloading Ollama model 'llama2'...")
# Use subprocess.run to wait for the download to complete
result = subprocess.run(["ollama", "pull", "llama2"], capture_output=True, text=True)
print("Ollama pull output:")
print(result.stdout)
print(result.stderr)

if result.returncode != 0:
    print("Error downloading model. Please check the model name and try again.")
else:
    print("Model 'llama2' downloaded successfully.")

Ollama pull output:

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 8934d96d3f08:   0% ▕                  ▏ 485 KB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   0% ▕                  ▏  15 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   1% ▕                  ▏  46 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   2% ▕                  ▏  75 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   3% ▕                  ▏ 102 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   3% ▕                  ▏ 130 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   4% ▕                  ▏ 158 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   4% ▕                  ▏ 169 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   5% ▕     

In [5]:
!pip install ollama --quiet
import ollama

# Configure the Ollama client
# Replace with the actual URL of your Ollama server if it's not running on the default host/port
# If running locally and need Colab access, you might need to use ngrok or similar.
client = ollama.Client(host='http://localhost:11434')

# This function is now used by the answer_question function.
def generate_answer_with_ollama(prompt, model="llama2"):
    """
    Generates a text completion using an Ollama model.

    Args:
        prompt (str): The input prompt for text generation.
        model (str): The name of the Ollama model to use (e.g., "llama2", "mistral").

    Returns:
        str: The generated text.
    """
    try:
        response = client.generate(
            model=model,
            prompt=prompt,
            stream=False # Set to True if you want streaming responses
        )
        return response['response']
    except ollama.ResponseError as e:
        print(f"Ollama API error: {e}")
        return f"Error generating answer with Ollama: {e}"
    except Exception as e:
        print(f"An unexpected error occurred during Ollama generation: {e}")
        return f"An unexpected error occurred: {e}"

# Note: The answer_question function (in cell k4A6TI6mzSG2) calls this function.

In [6]:
# Verify that Ollama is running and the model is available
print("Checking Ollama status and model list...")
result = subprocess.run(["ollama", "list"], capture_output=True, text=True)
print("Ollama list output:")
print(result.stdout)
print(result.stderr)

if "llama2" in result.stdout:
    print("Ollama is running and 'llama2' model is available.")
else:
    print("Ollama or 'llama2' model not found. Check previous steps.")

Checking Ollama status and model list...
Ollama list output:
NAME             ID              SIZE      MODIFIED      
llama2:latest    78e26419b446    3.8 GB    3 seconds ago    


Ollama is running and 'llama2' model is available.


## Indexing and Storage

This section covers generating numerical representations (embeddings) of the document content and storing them in a searchable vector database.

In [7]:
!pip install faiss-cpu --quiet
import faiss
import numpy as np

class VectorStore:
    def __init__(self):
        # Updated dimension to match the embedding model (sentence-transformers/multi-qa-mpnet-base-dot-v1)
        self.index = faiss.IndexFlatL2(768)
        self.texts = []

    def add(self, text, embedding):
        self.index.add(np.array([embedding]).astype('float32'))
        self.texts.append(text)

    def search(self, query_embedding, top_k=3):
        D, I = self.index.search(np.array([query_embedding]).astype('float32'), top_k)
        return [self.texts[i] for i in I[0]]

# Note: This class is used by process_document and answer_question.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.5 MB/s eta 0:00:00


In [8]:
!pip install sentence-transformers --quiet
from sentence_transformers import SentenceTransformer

# Load the sentence transformer model
# Using the same model as before for consistency with FAISS dimension
embedding_model = SentenceTransformer("sentence-transformers/multi-qa-mpnet-base-dot-v1")

def get_embedding(text):
    # Use the sentence_transformers library to get embeddings
    embedding = embedding_model.encode(text)
    return embedding.tolist() # Return as a list to match previous format if needed

# Note: This function is called by process_document and answer_question.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.3 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
store = VectorStore()

def process_document(file_input): # Renamed and changed parameter name for clarity
    print("--- Starting process_document function ---") # Debug print

    # Gradio File component provides a path via file_input.name
    file_path = file_input.name if file_input else None

    if not file_path:
        print("Error: No file uploaded.")
        return "Error: No file uploaded."

    pages = [] # Initialize pages list
    try:
        print(f"--- Calling load_document for {file_path} ---") # Debug print
        pages = load_document(file_path) # Call the new function
        print(f"--- load_document returned {len(pages) if pages else 0} pages ---") # Debug print

        # Debugging print: Show extracted text or indicate if extraction failed
        if not pages:
            print("Error: No text extracted from the document by load_document.")
            return "Error: No text extracted from the document by load_document."
        else:
            print(f"Successfully extracted text from {len(pages)} pages.")
            # Optionally print a snippet of the extracted text
            # print("Extracted text snippet (first 500 chars):")
            # print("".join(pages)[:500])
            # print("-" * 20)

    except Exception as e:
        print(f"Error during load_document for {file_path}: {e}")
        print("-" * 20)
        return f"Error during document loading: {e}"


    # Clear the store before adding new data from a new document
    print("--- Clearing/Initializing VectorStore ---") # Debug print
    global store
    store = VectorStore()
    print("--- VectorStore cleared/initialized ---") # Debug print


    print("--- Starting chunking and embedding loop ---") # Debug print
    for i, text in enumerate(pages):
        print(f"--- Processing page {i+1}/{len(pages)} ---") # Debug print
        chunks = chunk_text(text)
        print(f"--- Page {i+1} chunked into {len(chunks)} chunks ---") # Debug print
        for j, chunk in enumerate(chunks):
            # print(f"--- Embedding chunk {j+1}/{len(chunks)} of page {i+1} ---") # Too verbose?
            try:
                emb = get_embedding(chunk)
                # print(f"--- Embedding obtained for chunk {j+1} ---") # Too verbose?
                # print(f"--- Adding chunk {j+1} to VectorStore ---") # Too verbose?
                store.add(chunk, emb)
                # print(f"--- Chunk {j+1} added to VectorStore ---") # Too verbose?
            except Exception as e:
                print(f"Error embedding or adding chunk: {e}")
                # Continue processing other chunks even if one fails
                pass

    print("--- Chunking and embedding loop finished ---") # Debug print
    return "Document processed and ready!"

## RAG Pipeline Core (Retrieval & Generation)

This section covers both the **retrieval** of relevant document chunks from the vector store (using the VectorStore's search method) and the **generation** of the final answer. The generation uses the retrieved context and the user query to create a grounded answer using a language model (Ollama), implemented with Langchain Runnables and Prompt Templates.

In [11]:
def chunk_text(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

class VectorStore:
    def __init__(self):
        # Updated dimension to match the embedding model (sentence-transformers/multi-qa-mpnet-base-dot-v1)
        self.index = faiss.IndexFlatL2(768)  # embedding dim assumed 768, adjust if needed
        self.texts = []

    def add(self, text, embedding):
        self.index.add(np.array([embedding]).astype('float32'))
        self.texts.append(text)

    def search(self, query_embedding, top_k=5):
        D, I = self.index.search(np.array([query_embedding]).astype('float32'), top_k)
        return [self.texts[i] for i in I[0] if i != -1]

# Note: The get_embedding function is defined in cell rpOLYhSCzY7M
# The ollama client is initialized in cell 25286156
# The generate_answer_with_ollama function is defined in cell 25286156

# Import Langchain components for generation
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# Import ChatOllama from the new package
!pip install -U langchain-ollama --quiet # Already installed in previous step
from langchain_ollama import ChatOllama


# Define the Langchain components for the RAG chain
# Assuming Ollama server is running at localhost:11434 and model 'llama2' is pulled
# You might need to adjust model name and base_url if different
llm = ChatOllama(model="llama2", base_url="http://localhost:11434")

# Define the prompt template using Langchain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
{context}

Question: {question}
""")

# Create a RAG chain using Langchain Runnables
# This chain takes context and question, formats the prompt, and passes it to the LLM
rag_chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


def answer_question(question):
    print("--- Starting answer_question function (Langchain Runnables) ---")
    try:
        print("--- Getting question embedding ---")
        q_emb = get_embedding(question)
        print("--- Question embedding obtained ---")

        print("--- Searching vector store ---")
        # Increase top_k to retrieve more context
        context_docs = store.search(q_emb, top_k=5)
        # Join the retrieved text chunks
        context_text = "\n\n".join(context_docs)
        print("--- Vector store search completed ---")

        # Debugging prints
        print("Retrieved Context:")
        print(context_text)
        print("-" * 20)

        prompt = context_text + f"\n\nQuestion: {question}\nAnswer:" # Prompt created by Langchain template now

        print("Generated Prompt:")
        print(prompt)
        print("-" * 20)

        print("--- Invoking Langchain RAG chain ---")
        # Invoke the Langchain RAG chain with context and question
        response = rag_chain.invoke({"context": context_text, "question": question})
        print("--- Langchain RAG chain invocation completed ---")

        # Debugging print for successful response
        print("Model Response:")
        print(response)
        print("-" * 20)
        return response

    except Exception as outer_e:
        # Catch any other errors in the function
        print(f"An unexpected error occurred in answer_question: {outer_e}")
        print("-" * 20)
        return f"An unexpected error occurred: {outer_e}"

## User Interaction

This section covers setting up the user interface using Gradio to allow users to upload documents, process them, ask questions, and view answers.

In [13]:
!pip install gradio --quiet
import gradio as gr

with gr.Blocks() as demo:
    with gr.Row():
        pdf_input = gr.File(label="Upload Document") # Changed label
        upload_btn = gr.Button("Process Document") # Changed label
        status = gr.Textbox(label="Status")

    # Call the new process_document function
    upload_btn.click(process_document, inputs=[pdf_input], outputs=[status])

    question = gr.Textbox(label="Ask a question about the Document") # Changed label
    answer = gr.Textbox(label="Answer")

    question.submit(answer_question, inputs=[question], outputs=[answer])

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://919fa3e5eabac172d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
